## Did Azithromycin prevent death by coronary heart disease?

Within the past decade, serious attention has been given to the possibility of bacterial vectors as causal factors of atherosclerosis. To date, the greatest amount of information has related to the intracellular organism Chlamydia pneumoniae. This interest has been stimulated by the frequent finding of bacterial antigens and, occasionally, recoverable organisms, within human atherosclerotic plaque. Indirect evidence for and against the benefit of anti-Chlamydia antibiotic agents comes from epidemiologic studies.

Anderson JL, Muhlestein JB. Antibiotic trials for coronary heart disease. Tex Heart Inst J. 2004;31(1):33-8. PMID: 15061624

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import zipfile
import gzip
import io
import warnings
warnings.simplefilter('ignore')

# import dowhy
# from dowhy import CausalModel

from src import RWE

In [2]:
# %pwd
FILEPATH_patients = 'C:/Users/t-jongshin/Desktop/workstation/2021_rwe/api/asset/mimic-iv-1.0/core/patients.csv.gz'

FILEPATH_d_diagno = 'C:/Users/t-jongshin/Desktop/workstation/2021_rwe/api/asset/mimic-iv-1.0/hosp/d_icd_diagnoses.csv.gz'
FILEPATH_diagnose = 'C:/Users/t-jongshin/Desktop/workstation/2021_rwe/api/asset/mimic-iv-1.0/hosp/diagnoses_icd.csv.gz'

FILEPATH_microbio = 'C:/Users/t-jongshin/Desktop/workstation/2021_rwe/api/asset/mimic-iv-1.0/hosp/microbiologyevents.csv.gz'
FILEPATH_pharmacy = 'C:/Users/t-jongshin/Desktop/workstation/2021_rwe/api/asset/mimic-iv-1.0/hosp/pharmacy.csv.gz'

In [3]:
# zf = zipfile.ZipFile(FILEPATH, 'r')
# zf.namelist()[3]

# with io.BytesIO(zf.read(zf.namelist()[3])) as f:
#     df = f.read()

In [4]:
# %%timeit -n 1 -r 1

with gzip.open(FILEPATH_patients, 'rb') as f:    
    dat_patients = pd.read_csv(f)

with gzip.open(FILEPATH_diagnose, 'rb') as f:    
    dat_diagnose = pd.read_csv(f)

# with gzip.open(FILEPATH_microbio, 'rb') as f:    
#     dat_microbio = pd.read_csv(f)

with gzip.open(FILEPATH_pharmacy, 'rb') as f:    
    dat_pharmacy = pd.read_csv(f)

In [5]:
with gzip.open(FILEPATH_d_diagno, 'rb') as f:    
    dic_diagnose = pd.read_csv(f)

In [6]:
display(dat_patients.iloc[0:1,:])
display(dat_diagnose.iloc[0:1,:])
# display(dat_microbio.iloc[0:1,:])
display(dat_pharmacy.iloc[0:1,:])

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000048,F,23,2126,2008 - 2010,NaN


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,15734973,20475282,3,2825,9


,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,17868682,22726960,2697460,17868682-19,2160-01-07 14:00:00,2160-01-07 14:00:00,NaN,IV Large Volume,Discontinued,2160-01-07 13:06:18,...,NaN,NaN,NaN,NaN,Ongoing,NaN,Enter on Label,NaN,Distribution-Floor Stock,NaN


## Filter patients by any conditions associated with coronary syndromes

In [7]:
dic_diagnose['long_title'] = dic_diagnose['long_title'].str.lower()

In [8]:
lst_icd = dic_diagnose[dic_diagnose['long_title'].str.find('coronary') >= 0]
lst_icd

,icd_code,icd_version,long_title
4712,4111,9,intermediate coronary syndrome
4713,41181,9,acute coronary occlusion without myocardial in...
4719,41400,9,coronary atherosclerosis of unspecified type o...
4720,41401,9,coronary atherosclerosis of native coronary ar...
4721,41402,9,coronary atherosclerosis of autologous vein by...
...,...,...,...
99666,V4581,9,aortocoronary bypass status
99667,V4582,9,percutaneous transluminal coronary angioplasty...
109648,Z951,10,presence of aortocoronary bypass graft
109652,Z955,10,presence of coronary angioplasty implant and g...


In [9]:
lst_icd = lst_icd[['icd_code']]

### Merging list of icd code with the list of patients

In [10]:
temp = pd.merge(dat_diagnose, lst_icd, how='inner', on='icd_code')[['subject_id']].drop_duplicates()
dat_patients_disease = pd.merge(temp, dat_patients, on='subject_id')
dat_patients_disease

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,17872922,M,88,2131,2008 - 2010,2141-03-27
1,12201410,M,75,2126,2008 - 2010,NaN
2,12849817,M,65,2166,2008 - 2010,NaN
3,12292383,M,75,2179,2008 - 2010,NaN
4,18739672,M,71,2113,2008 - 2010,NaN
...,...,...,...,...,...,...
35101,15335760,M,33,2141,2014 - 2016,2144-08-05
35102,17176505,M,79,2160,2008 - 2010,NaN
35103,18353395,M,73,2140,2017 - 2019,NaN
35104,17177034,M,75,2153,2008 - 2010,NaN


In [11]:
# list of all-cause death with the coronary syndrome associated conditions
dat_patients_disease_id = []
dat_patients_disease_id.append(dat_patients_disease[dat_patients_disease['dod'].isna()])#[['subject_id']]
dat_patients_disease_id.append(dat_patients_disease[~dat_patients_disease['dod'].isna()])#[['subject_id']]
display(dat_patients_disease_id[0].head(5))
display(dat_patients_disease_id[1].head(5))

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
1,12201410,M,75,2126,2008 - 2010,NaN
2,12849817,M,65,2166,2008 - 2010,NaN
3,12292383,M,75,2179,2008 - 2010,NaN
4,18739672,M,71,2113,2008 - 2010,NaN
6,13864194,M,71,2135,2008 - 2010,NaN


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,17872922,M,88,2131,2008 - 2010,2141-03-27
5,14166890,M,64,2161,2008 - 2010,2161-12-17
11,13976907,F,66,2159,2008 - 2010,2169-02-21
12,13108883,M,54,2121,2008 - 2010,2124-03-09
16,16145193,F,60,2157,2008 - 2010,2167-12-28


## Filter pharmacy list by any drug analog of Azithromycin

In [12]:
# array(['Azithromycin', 'Clarithromycin', 'Erythromycin 0.5% Ophth Oint',
#        'Erythromycin', 'Erythromycin Ethylsuccinate Suspension',
#        'erythromycin with ethanol', 'Erythromycin Tpoical Gel',
#        'erythromycin', 'azithromycin', 'Neo*PO*Azithromycin',
#        'Erythromycin opthalmic', 'Erythromycin Ethylsuccinate',
#        'clarithromycin', 'Erythromycin ', 'azithromycin PO Susp'],
#       dtype=object)

In [13]:
# list of drug names containing thromycin
search_idx = dat_pharmacy['medication'].str.find('thromycin')
dat_pharmacy[search_idx >= 0]['medication'].unique() #NA = -1.0

array(['Azithromycin', 'Clarithromycin', 'Erythromycin 0.5% Ophth Oint',
       'Erythromycin', 'Erythromycin Ethylsuccinate Suspension',
       'erythromycin with ethanol', 'Erythromycin Tpoical Gel',
       'erythromycin', 'azithromycin', 'Neo*PO*Azithromycin',
       'Erythromycin opthalmic', 'Erythromycin Ethylsuccinate',
       'clarithromycin', 'Erythromycin ', 'azithromycin PO Susp'],
      dtype=object)

In [14]:
# search results for patients who were prescribed the drugs
search_result_pos = dat_pharmacy[search_idx >= 0]
search_result_pos.shape

In [15]:
search_result_pos.head(5)

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
141,12315540,23248664,92297162,12315540-327,2172-12-03 23:00:00,2172-12-04 12:00:00,Azithromycin,IV Piggyback,Discontinued,2172-12-03 22:49:47,...,NaN,NaN,1.0,NaN,Ongoing,NaN,Enter on Label,NaN,IV MED,NaN
817,17067646,25423329,27908492,17067646-288,2157-09-03 11:00:00,2157-09-16 20:00:00,Clarithromycin,Unit Dose,Discontinued via patient discharge,2157-09-03 10:37:24,...,NaN,NaN,2.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
905,17067646,25326346,4146153,17067646-449,2157-09-19 04:00:00,2157-09-21 14:00:00,Azithromycin,IV Piggyback,Discontinued,2157-09-19 03:06:37,...,NaN,NaN,1.0,NaN,Ongoing,NaN,Enter on Label,NaN,IV MED,NaN
930,17067646,25326346,34910737,17067646-535,2157-09-21 15:00:00,2157-09-21 17:00:00,Azithromycin,Unit Dose,Discontinued,2157-09-21 14:49:44,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
932,17067646,25326346,70146485,17067646-546,2157-09-22 12:00:00,2157-09-24 09:00:00,Azithromycin,Unit Dose,Discontinued,2157-09-22 11:22:38,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN


In [16]:
# search results for patients who were NOT prescribed the drugs
search_result_neg = dat_pharmacy[search_idx == -1]
search_result_neg.shape

In [17]:
search_result_neg.head(5)

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
1,17868682,22726960,1383959,17868682-20,2160-01-07 15:00:00,2160-01-08 16:00:00,Pneumococcal Vac Polyvalent,Unit Dose,Discontinued via patient discharge,2160-01-07 14:25:45,...,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
2,17868682,22726960,64138666,17868682-23,2160-01-07 15:00:00,2160-01-08 16:00:00,Sodium Chloride 0.9% Flush,Unit Dose,Discontinued via patient discharge,2160-01-07 14:25:45,...,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Floor Stock Item,NaN
3,17868682,22726960,43535627,17868682-28,2160-01-07 15:00:00,2160-01-08 14:00:00,Ondansetron,Unit Dose,Expired,2160-01-07 14:25:45,...,NaN,NaN,1.0,1.0,Doses,36.0,Hours,NaN,Omnicell,NaN
4,17868682,22726960,13072826,17868682-29,2160-01-07 15:00:00,2160-01-07 16:00:00,Morphine Sulfate,Unit Dose,Discontinued,2160-01-07 14:25:45,...,0.0,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
5,17868682,22726960,16634804,17868682-30,2160-01-07 10:00:00,2160-01-07 16:00:00,Aspirin,Unit Dose,Discontinued,2160-01-07 14:25:45,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN


## Deceased patients who has conditions associated coronary syndrome and also has taken Azithromycin analogs
O | T = 1

In [20]:
dod_treated = []
dod_treated.append(pd.merge(dat_patients_disease_id[0], search_result_pos, how='inner', on='subject_id').drop_duplicates()) #dod == 0
dod_treated.append(pd.merge(dat_patients_disease_id[1], search_result_pos, how='inner', on='subject_id').drop_duplicates()) #dod == 1
dod_treated[0].shape, dod_treated[0]['subject_id'].drop_duplicates().shape, dod_treated[1].shape, dod_treated[1]['subject_id'].drop_duplicates().shape

In [21]:
display(dod_treated[0].head(5))
display(dod_treated[1].head(5))

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,pharmacy_id,poe_id,starttime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,14267880,M,73,2135,2008 - 2010,NaN,25295827,57526848,14267880-3140,2146-05-29 01:00:00,...,NaN,NaN,1.0,1.0,Doses,365.0,Days,NaN,Omnicell,NaN
1,14267880,M,73,2135,2008 - 2010,NaN,25295827,42983361,14267880-3141,2146-05-30 01:00:00,...,NaN,NaN,1.0,4.0,Doses,365.0,Days,NaN,Omnicell,NaN
2,10669695,F,88,2153,2008 - 2010,NaN,21732489,67565933,10669695-626,2158-10-30 05:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
3,10669695,F,88,2153,2008 - 2010,NaN,23053402,11679826,10669695-736,2159-07-29 17:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
4,10669695,F,88,2153,2008 - 2010,NaN,23053402,82391224,10669695-849,2159-08-01 17:00:00,...,NaN,NaN,1.0,5.0,Doses,36.0,Hours,NaN,Omnicell,NaN


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,pharmacy_id,poe_id,starttime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,13976907,F,66,2159,2008 - 2010,2169-02-21,28205831,12986587,13976907-989,2158-12-15 10:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
1,13976907,F,66,2159,2008 - 2010,2169-02-21,28205831,72641268,13976907-1020,2158-12-16 11:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
2,13976907,F,66,2159,2008 - 2010,2169-02-21,28205831,62319538,13976907-1063,2158-12-18 10:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
3,13976907,F,66,2159,2008 - 2010,2169-02-21,21402581,36648906,13976907-3308,2168-05-08 15:00:00,...,NaN,NaN,1.0,NaN,Ongoing,365.0,Days,NaN,Omnicell,NaN
4,13976907,F,66,2159,2008 - 2010,2169-02-21,25735538,28647822,13976907-4566,2169-02-09 01:00:00,...,NaN,NaN,1.0,1.0,Doses,365.0,Days,NaN,Omnicell,NaN


In [22]:
dod_treated[0].columns 

Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod', 'hadm_id', 'pharmacy_id', 'poe_id',
       'starttime', 'stoptime', 'medication', 'proc_type', 'status',
       'entertime', 'verifiedtime', 'route', 'frequency', 'disp_sched',
       'infusion_type', 'sliding_scale', 'lockout_interval', 'basal_rate',
       'one_hr_max', 'doses_per_24_hrs', 'duration', 'duration_interval',
       'expiration_value', 'expiration_unit', 'expirationdate', 'dispensation',
       'fill_quantity'],
      dtype='object')

## Deceased patients who has conditions associated coronary syndrome and has NOT taken Azithromycin analogs
O | T = 0

In [24]:
dod_not_treated = []

for i in range(2):
    dod_not_treated.append(pd.merge(dat_patients_disease_id[i], search_result_pos, how='outer', on='subject_id', indicator=True).drop_duplicates())
    dod_not_treated_idx = dod_not_treated[i][dod_not_treated[i]['_merge']!='both'][['subject_id']].drop_duplicates()#.iloc[:,:-1]

    dod_not_treated[i] = pd.merge(dod_not_treated_idx, dat_pharmacy, how='inner', on='subject_id').drop_duplicates()

dod_not_treated[0].shape, dod_not_treated[0]['subject_id'].drop_duplicates().shape, dod_not_treated[1].shape, dod_not_treated[1]['subject_id'].drop_duplicates().shape

In [25]:
display(dod_not_treated[0].head(5))
display(dod_not_treated[1].head(5))

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,12201410,20381641,44752634,12201410-176,2126-08-12 16:00:00,2126-08-13 15:00:00,Miconazole Powder 2%,Unit Dose,Discontinued via patient discharge,2126-08-12 15:36:40,...,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Bulk Item,NaN
1,12201410,20381641,44610849,12201410-148,2126-08-12 18:00:00,2126-08-13 15:00:00,Influenza Virus Vaccine,Unit Dose,Discontinued via patient discharge,2126-08-12 17:31:41,...,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
2,12201410,20381641,74650754,12201410-152,2126-08-12 10:00:00,2126-08-13 15:00:00,Aspirin,Unit Dose,Discontinued via patient discharge,2126-08-12 17:31:41,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
3,12201410,20381641,83563801,12201410-153,2126-08-12 10:00:00,2126-08-13 15:00:00,Atorvastatin,Unit Dose,Discontinued via patient discharge,2126-08-12 17:31:41,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
4,12201410,20381641,868580,12201410-154,2126-08-12 10:00:00,2126-08-13 15:00:00,Fludrocortisone Acetate,Unit Dose,Discontinued via patient discharge,2126-08-12 17:31:41,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN


,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,17872922,27151133,54888785,17872922-767,2131-12-09 16:00:00,2131-12-10 14:00:00,Influenza Virus Vaccine,Unit Dose,Discontinued via patient discharge,2131-12-09 15:09:50,...,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
1,17872922,27151133,71056155,17872922-769,2131-12-09 16:00:00,2131-12-10 14:00:00,Sodium Chloride 0.9% Flush,Unit Dose,Discontinued via patient discharge,2131-12-09 15:09:50,...,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Floor Stock Item,NaN
2,17872922,27151133,32844426,17872922-775,2131-12-09 16:00:00,2131-12-10 06:00:00,NaN,IV Large Volume,Discontinued,2131-12-09 15:09:50,...,NaN,NaN,NaN,NaN,Ongoing,NaN,Enter on Label,NaN,Distribution-Floor Stock,NaN
3,17872922,27151133,65683902,17872922-776,2131-12-09 16:00:00,2131-12-10 14:00:00,Morphine Sulfate,Unit Dose,Discontinued via patient discharge,2131-12-09 15:09:50,...,0.0,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
4,17872922,27151133,97717632,17872922-776,2131-12-09 16:00:00,2131-12-10 14:00:00,Morphine Sulfate,Unit Dose,Discontinued via patient discharge,2131-12-09 15:09:50,...,0.0,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN


In [26]:
# making sure there is no overlap
dod_overlap = pd.merge(dod_treated[0][['subject_id']].drop_duplicates(), dod_not_treated[0][['subject_id']].drop_duplicates(), how='inner', on='subject_id')

In [27]:
dod_overlap.shape

In [28]:
display(dod_treated[0]['medication'])
display(dod_not_treated[0]['medication'])

0       Azithromycin
1       Azithromycin
2       Azithromycin
3       Azithromycin
4       Azithromycin
            ...     
9624    Erythromycin
9625    Azithromycin
9626    Azithromycin
9627    Azithromycin
9628    Azithromycin
Name: medication, Length: 9629, dtype: object

0                                       Miconazole Powder 2%
1                                    Influenza Virus Vaccine
2                                                    Aspirin
3                                               Atorvastatin
4                                    Fludrocortisone Acetate
                                 ...                        
5928159                                 Sucrose Solution 24%
5928160                         Erythromycin 0.5% Ophth Oint
5928161    Phytonadione (Vitamin K1) (for L&D to NBN orde...
5928162                                  Hepatitis B Vaccine
5928163                                          Glucose Gel
Name: medication, Length: 5928164, dtype: object

In [29]:
# _,b,c = np.intersect1d(dod_treated[0].columns, dod_not_treated[0].columns, return_indices=True)

In [30]:
# dod_treated[0].columns[[i for i in range(32) if i not in b]]

### Binarize treatment and outcome

In [31]:
for i in range(2): #dod
    for j in range(2): #med
        if j == 1:
            dod_treated[i]['dod'] = i
            dod_treated[i]['medication'] = j
        else:
            dod_not_treated[i]['dod'] = i
            dod_not_treated[i]['medication'] = j

In [32]:
display(dod_treated[1].head(5))
display(dod_not_treated[0].head(5))

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,pharmacy_id,poe_id,starttime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,13976907,F,66,2159,2008 - 2010,1,28205831,12986587,13976907-989,2158-12-15 10:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
1,13976907,F,66,2159,2008 - 2010,1,28205831,72641268,13976907-1020,2158-12-16 11:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
2,13976907,F,66,2159,2008 - 2010,1,28205831,62319538,13976907-1063,2158-12-18 10:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
3,13976907,F,66,2159,2008 - 2010,1,21402581,36648906,13976907-3308,2168-05-08 15:00:00,...,NaN,NaN,1.0,NaN,Ongoing,365.0,Days,NaN,Omnicell,NaN
4,13976907,F,66,2159,2008 - 2010,1,25735538,28647822,13976907-4566,2169-02-09 01:00:00,...,NaN,NaN,1.0,1.0,Doses,365.0,Days,NaN,Omnicell,NaN


,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity,dod
0,12201410,20381641,44752634,12201410-176,2126-08-12 16:00:00,2126-08-13 15:00:00,0,Unit Dose,Discontinued via patient discharge,2126-08-12 15:36:40,...,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Bulk Item,NaN,0
1,12201410,20381641,44610849,12201410-148,2126-08-12 18:00:00,2126-08-13 15:00:00,0,Unit Dose,Discontinued via patient discharge,2126-08-12 17:31:41,...,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN,0
2,12201410,20381641,74650754,12201410-152,2126-08-12 10:00:00,2126-08-13 15:00:00,0,Unit Dose,Discontinued via patient discharge,2126-08-12 17:31:41,...,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN,0
3,12201410,20381641,83563801,12201410-153,2126-08-12 10:00:00,2126-08-13 15:00:00,0,Unit Dose,Discontinued via patient discharge,2126-08-12 17:31:41,...,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN,0
4,12201410,20381641,868580,12201410-154,2126-08-12 10:00:00,2126-08-13 15:00:00,0,Unit Dose,Discontinued via patient discharge,2126-08-12 17:31:41,...,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN,0


In [33]:
dod_treated[0].columns

Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod', 'hadm_id', 'pharmacy_id', 'poe_id',
       'starttime', 'stoptime', 'medication', 'proc_type', 'status',
       'entertime', 'verifiedtime', 'route', 'frequency', 'disp_sched',
       'infusion_type', 'sliding_scale', 'lockout_interval', 'basal_rate',
       'one_hr_max', 'doses_per_24_hrs', 'duration', 'duration_interval',
       'expiration_value', 'expiration_unit', 'expirationdate', 'dispensation',
       'fill_quantity'],
      dtype='object')

### Combine four dataframes into one master micro data

In [34]:
microdf = pd.concat([dod_treated[0], dod_treated[1]])
microdf = pd.concat([microdf, dod_not_treated[0]])
microdf = pd.concat([microdf, dod_not_treated[1]])

In [35]:
microdf[['medication']] = microdf[['medication']].astype(bool)

In [36]:
microdf['medication']

0           True
1           True
2           True
3           True
4           True
           ...  
4199187    False
4199188    False
4199189    False
4199190    False
4199191    False
Name: medication, Length: 10139277, dtype: bool

In [37]:
# stop

In [100]:
tempdf = microdf.sample(50000)

In [101]:
# tempdf = tempdf[['gender', 'dod', 'medication']].reset_index(drop=True)
tempdf = tempdf.reset_index(drop=True)

In [102]:
tempdf

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,pharmacy_id,poe_id,starttime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,16286648,NaN,NaN,NaN,NaN,1,27089181,23079391,16286648-282,2186-02-17 10:00:00,...,NaN,NaN,1.0,NaN,Ongoing,365.0,Days,NaN,Omnicell,NaN
1,11151938,NaN,NaN,NaN,NaN,0,25334129,6825142,11151938-1344,2158-10-16 20:00:00,...,NaN,NaN,2.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
2,15790932,NaN,NaN,NaN,NaN,0,21167646,94749511,15790932-92,2113-04-17 17:00:00,...,0.0,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,OmniCell,NaN
3,15374189,NaN,NaN,NaN,NaN,0,20817438,11866162,15374189-22,2164-05-11 10:00:00,...,NaN,NaN,1.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
4,19894790,NaN,NaN,NaN,NaN,0,27933693,65909495,19894790-406,2140-09-11 14:00:00,...,NaN,NaN,0.0,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,16151467,NaN,NaN,NaN,NaN,0,26232518,3034750,16151467-451,2178-10-23 01:00:00,...,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
49996,15251725,NaN,NaN,NaN,NaN,1,29931688,75746209,15251725-32,2110-07-22 07:00:00,...,NaN,NaN,1.0,1.0,Doses,NaN,Enter on Label,NaN,OmniCell,NaN
49997,14028368,NaN,NaN,NaN,NaN,1,28525642,97162754,14028368-329,2146-03-08 00:00:00,...,NaN,NaN,1.0,NaN,Ongoing,365.0,Days,NaN,Omnicell,NaN
49998,11535886,NaN,NaN,NaN,NaN,1,24504360,12918930,11535886-845,2177-03-02 22:00:00,...,NaN,NaN,1.0,NaN,Ongoing,365.0,Days,NaN,Omnicell,NaN


In [83]:
# dag = {D [outcome,pos="1.400,1.621"],
# E [exposure,pos="-2.200,1.597"],
# E -> D
# }

In [84]:
# 'digraph {         treatment[label="Program Signup in month i"];         pre_spends;         post_spends;         Z->treatment;         U[label="Unobserved Confounders"];         pre_spends -> treatment;         treatment->post_spends;         signup_month->post_spends; signup_month->pre_spends;         signup_month->treatment;         U->treatment; U->pre_spends; U->post_spends;         }'

In [103]:
SW = RWE.rwe_inference.showWhy(df=tempdf, treatment='medication', outcome='dod', common_causes='gender', graph=None)

In [104]:
SW.dowhy_identify()

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                               
─────────────(Expectation(dod|gender))
d[medication]                         
Estimand assumption 1, Unconfoundedness: If U→{medication} and U→dod then P(dod|medication,gender,U) = P(dod|medication,gender)

### Estimand : 2
Estimand name: iv
No such variable found!

### Estimand : 3
Estimand name: frontdoor
No such variable found!



In [105]:
SW.dowhy_estimate('psm', 'att')

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                               
─────────────(Expectation(dod|gender))
d[medication]                         
Estimand assumption 1, Unconfoundedness: If U→{medication} and U→dod then P(dod|medication,gender,U) = P(dod|medication,gender)

## Realized estimand
b: dod~medication+gender
Target units: att

## Estimate
Mean value: 0.18333333333333332



In [107]:
SW.dowhy_refute(linear=False)

binary_flip binary_flip
Refute: Add a Random Common Cause
Estimated effect:0.18333333333333332
New effect:-0.6333333333333333

Refute: Add an Unobserved Common Cause
Estimated effect:0.18333333333333332
New effect:0.36427320490367776

Refute: Use a Placebo Treatment
Estimated effect:0.18333333333333332
New effect:0.4
p value:0.0

Refute: Use a subset of data
Estimated effect:0.18333333333333332
New effect:0.20026481297583582
p value:0.010035980100274105

